# Back-Translation
- Back-translate each of the training files in 4 languages and save them as txts.

In [1]:
import sys; sys.path.append('../../genre-bias/traditional-machine-learning/')
import data_loader
import pickle
import os
from googletrans import Translator

with open('../../../train_fnames_scenario_dict_exp_1.pickle', 'rb') as f:
    TRAIN_FNAMES = pickle.load(f)
    
translator = Translator()

EDA Parameters: N = 25 | alpha = 0.05


In [2]:
def back_translate(text, lang):
    """
    Translates the given English text to 'lang'. Translates that back to English.
    Returns the translated English text.
    """
    first = translator.translate(text, src='en', dest=lang)
    back = translator.translate(first.text, src=lang, dest='en')
    return back.text

In [3]:
languages = ['fr', 'ko', 'de', 'es'] # French, Korean, German, Spanish

for scenario in ['A', 'B', 'C', 'D']:
    fiction_fnames = TRAIN_FNAMES[scenario]['fiction_fnames']
    non_fiction_fnames = TRAIN_FNAMES[scenario]['non_fiction_fnames']

    print("\n\n\nTotal files for Case {} = {}".format(scenario, len(fiction_fnames+non_fiction_fnames)))
    
    for fname in fiction_fnames:
        for lang in languages:
            text = data_loader.get_passage(fname)
            name = fname.split('/')[-1][:-4] + "__fic__lang_" + lang + ".txt"
            print(name)
            name_path = '../../../data/back-translated/'+name
            if os.path.isfile(name_path):
                print("---------Already exists:", name_path)
                continue
                
            translated = back_translate(text, lang)
            with open(name_path, 'w') as f:
                f.write(translated)
                
    print("Done with fiction!")
        
    for fname in non_fiction_fnames: # need two "passages" per txt
        for lang in languages:
            two_texts = data_loader.get_passage(fname, two_passages=True)
            for i in [0,1]: # back-translate and write both passages
                name = fname.split('/')[-1][:-4] + "__" + str(i+1) + "__non__lang_" + lang + ".txt"
                print(name)
                name_path = '../../../data/back-translated/'+name
                if os.path.isfile(name_path):
                    print("----Already exists:", name_path)
                    continue
                    
                translated = back_translate(two_texts[i], lang)
                with open(name_path, 'w') as f:
                    f.write(translated)
                    
    print("Done with non-fiction!")

In [1]:
# Sanity check: check that we have back-translations for one and all

import pickle
import os

with open('../../../train_fnames_scenario_dict_exp_1.pickle', 'rb') as f:
    TRAIN_FNAMES = pickle.load(f)

outs = []
for c in TRAIN_FNAMES:
    for dic in TRAIN_FNAMES[c]:
        outs.extend(TRAIN_FNAMES[c][dic])
        
cmp1 = []
for i in outs:
    cmp1.append(i.split('/')[-1][:-4])
    
print(len(cmp1))
print(len(set(cmp1)))

1201
584


In [2]:
names = []

for i in os.listdir('../../../data/back-translated/'):
    names.append(i.split('__')[0])
    
print(len(names))
print(len(set(names)))

2736
584


In [3]:
set(names) == set(cmp1)
# Great!

True